In [1]:
%%capture
!pip install langchain==0.1.4 openai==1.10.0 langchain-openai tiktoken faiss-cpu

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


#🔍 **Example Selectors in Langchain**

These are the brains picking the best examples for your prompts.

**Key Functions**:

- 🧠 **Smart Selection**: They choose which examples to use, based on the current input.

- 🎯 **Strategies**: Different methods, like looking for semantic similarity or ensuring diversity.

**Types**:

- **Semantic Similarity**: Finds examples closest in meaning to the input.

- **Maximal Marginal Relevance**: A mix of similarity and variety.

- **Prompt Length**: Picks examples to fit a certain length.

**Benefits**:

- ✨ **Dynamic**: Picks examples on-the-fly, not just a fixed list.

- 📏 **Manageable Prompts**: Keeps prompts concise and relevant.

**Customization**:
- 🛠️ Create your own by inheriting from `BaseExampleSelector`.

---

📜 **BaseExampleSelector**:
- 🗂️ The backbone class that all selectors inherit from.

- 🚀 Just define `select_examples` to decide how to pick the best examples.

---

🎲 **Creating a Random Example Selector**:

- **Add & Select**: It should be able to store examples and then pick randomly when prompted.

- **Custom Touch**: Craft it to fit your unique prompt crafting needs.

This is like teaching your AI to pick the right stories to tell at a party – it's all about relevance and variety!

In [3]:
from langchain.prompts.example_selector.base import BaseExampleSelector
from typing import Dict, List
import numpy as np

class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples: List[Dict[str, str]]):
        self.examples = examples

    def add_example(self, example: Dict[str, str]) -> None:
        """Add a new example to the list."""
        self.examples.append(example)

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """Randomly select examples based on the provided inputs."""
        return list(np.random.choice(self.examples, size=2, replace=False))

In [4]:
examples = [
    {"recipe": "Spaghetti Carbonara"},
    {"recipe": "Chicken Alfredo"},
    {"recipe": "Vegetable Stir Fry"}
]

In [5]:
# initialize an example selector
example_selector = CustomExampleSelector(examples)

### Example Placeholder:

🗂️ The {"recipe": "recipe"} is just a stand-in to show how the method works—it's not part of the selection process.

🛠️ In real use, this would be where the selector looks at the input to make smart choices.

But in the provided example, it's not utilized.



In [6]:
# randomly select some examples
example_selector.select_examples({"recipe": "recipe"})

[{'recipe': 'Spaghetti Carbonara'}, {'recipe': 'Vegetable Stir Fry'}]

In [7]:
# Add a new recipe to the collection
example_selector.add_example({"recipe": "Beef Stroganoff"})
print(example_selector.examples)

[{'recipe': 'Spaghetti Carbonara'}, {'recipe': 'Chicken Alfredo'}, {'recipe': 'Vegetable Stir Fry'}, {'recipe': 'Beef Stroganoff'}]


In [8]:
# Randomly select two recipes again
example_selector.select_examples({"recipe": "recipe"})

[{'recipe': 'Chicken Alfredo'}, {'recipe': 'Beef Stroganoff'}]

# Select by length

Ready for more?

Because below you will implement a `LengthBasedExampleSelector` which selects examples based on their length.

This is particularly useful when you're worried about the total length of the constructed prompt, especially given the context window limitations of some models.

In [23]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {"input": "caffeinated", "output": "sleepy"},
    {"input": "spooky", "output": "cuddly"},
    {"input": "crispy", "output": "soggy"},
    {"input": "galactic", "output": "mundane"},
    {"input": "funky", "output": "plain-jane"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "mysterious", "output": "transparent"},
    {"input": "delicious", "output": "tasteless"},
    {"input": "extravagant", "output": "simple"},
    {"input": "exhilarating", "output": "dull"},
    {"input": "mesmerizing", "output": "uninteresting"},
    {"input": "charismatic", "output": "ordinary"},
    {"input": "eccentric", "output": "conventional"},
    {"input": "adventurous", "output": "cautious"},
    {"input": "breathtaking", "output": "unremarkable"},
]

In [24]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template = "Input: {input}\nOutput: {output}"
)

# 📏 **LengthBasedExampleSelector Explained**

- **Purpose**: Chooses examples to avoid exceeding the model's context limit.
- **Process**:
   - **Addition**: When examples are added, their lengths are measured and noted.
   - **Selection**: When it's time to select, it picks examples that fit within the max prompt length.

**How It Operates**:
- 🧮 It calculates space left after considering the input length.
- ➕ Adds examples until there's no room left, aiming for the perfect fit.

In a nutshell, this selector is like a tailor, ensuring the content fits neatly within the model's capacity.


In [11]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt= example_prompt,
    max_length=15,
)

In [12]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="What's the opposite of...",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [13]:
long_string = "fluffy and puffy and cloud-like and soft as a marshmallow and as cuddly as a teddy bear in a world of cotton candy"
print(dynamic_prompt.format(adjective=long_string))

What's the opposite of...

Input: fluffy and puffy and cloud-like and soft as a marshmallow and as cuddly as a teddy bear in a world of cotton candy
Output:


In [14]:
short_string = "feeling wired"
print(dynamic_prompt.format(adjective=short_string))

What's the opposite of...

Input: caffeinated
Output: sleepy

Input: spooky
Output: cuddly

Input: crispy
Output: soggy

Input: feeling wired
Output:


# Selecting based on Maximal Marginal Relevance (MMR)

🔍 **Maximal Marginal Relevance (MMR) Selection**:

- **Goal**: Find a balance in example selection between relevance to the input and diversity among choices.

- **Relevance**: Measure how closely each example matches the input query.

- **Diversity**: Assess how different each example is from others already chosen.

- **MMR Calculation**: Combine relevance and diversity scores to decide each example's overall value. The formula is typically: $$ MMR = \lambda \times Relevance - (1 - \lambda) \times Diversity $$
   

- **Selection**: Pick the example with the best MMR score and keep going until you hit your limit or quality drops off.

MMR is like a curator artfully choosing exhibits for a gallery, ensuring each piece is compelling but not too similar to the next.


In [25]:
from langchain.prompts.example_selector import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)

from langchain_openai import OpenAIEmbeddings

from langchain_community.vectorstores import FAISS

from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [26]:
examples

[{'input': 'caffeinated', 'output': 'sleepy'},
 {'input': 'spooky', 'output': 'cuddly'},
 {'input': 'crispy', 'output': 'soggy'},
 {'input': 'galactic', 'output': 'mundane'},
 {'input': 'funky', 'output': 'plain-jane'},
 {'input': 'energetic', 'output': 'lethargic'},
 {'input': 'mysterious', 'output': 'transparent'},
 {'input': 'delicious', 'output': 'tasteless'},
 {'input': 'extravagant', 'output': 'simple'},
 {'input': 'exhilarating', 'output': 'dull'},
 {'input': 'mesmerizing', 'output': 'uninteresting'},
 {'input': 'charismatic', 'output': 'ordinary'},
 {'input': 'eccentric', 'output': 'conventional'},
 {'input': 'adventurous', 'output': 'cautious'},
 {'input': 'breathtaking', 'output': 'unremarkable'}]

In [27]:
example_prompt

PromptTemplate(input_variables=['input', 'output'], template='Input: {input}\nOutput: {output}')

## `MaxMarginalRelevanceExampleSelector` Overview

Balances relevance & diversity in example selection.

🔍 **Key Features:**

1. **Semantic Similarity Foundation**:

   - Uses semantic similarity for initial example selection.

2. **Important Attribute**:

   - `fetch_k`: Fetches a set number of examples before MMR reranking.

3. **Methods**:
   - `select_examples(input_variables)`: Creates query, fetches, and reranks examples using MMR.
   
   - `from_examples(...)`: Class method to initialize with examples, embeddings, and setup vector store.

In [71]:
mmr_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=2,
    fetch_k=5,
)

In [29]:
mmr_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=mmr_selector,
    example_prompt=example_prompt,
    prefix="What's the opposite of...",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [30]:
print(mmr_prompt.format(adjective="buzzed"))

What's the opposite of...

Input: exhilarating
Output: dull

Input: caffeinated
Output: sleepy

Input: buzzed
Output:


## 🔍 **`NGramOverlapExampleSelector`**

Focuses on n-gram overlap for example selection.

What is an n-gram?

Consider the following sentence: "Sunsets are always beautiful."

1️⃣ **Unigram (1-gram)**: Individual words. Example: "Sunsets," "are," "always," "beautiful".

2️⃣ **Bigram (2-gram)**: Pairs of words. "Sunsets are," "are always," "always beautiful".

3️⃣ **Trigram (3-gram)**: Trio of words. In this case, "Sunsets are always" and "are always beautiful."

4️⃣ **Fourgram (4-gram)**: A sequence of four words. With "Sunsets are always beautiful," the fourgram is "Sunsets are always very."

This method relies on `sentence_bleu` score for overlap calculation.

🧩 **Key Components:**
1. **N-gram Overlap Score**:
   - Uses `sentence_bleu` for n-gram overlap scoring.
   - Score ranges from 0.0 to 1.0.

2. **Important Attributes**:
   - `examples`: Stores a list of available examples.
   - `example_prompt`: Template for formatting examples.
   - `threshold`: Cut-off for example selection.

3. **Essential Methods**:
   - `add_example(example)`: Adds new examples to the list.
   - `select_examples(input_variables)`: Sorts and selects examples based on n-gram overlap.

🚀 **How It Works**:
- Computes n-gram overlap score for each example.
- Selects examples in descending score order until hitting the threshold.
- Useful for finding examples with common n-grams to the input.

In [74]:
from langchain.prompts.example_selector.ngram_overlap import NGramOverlapExampleSelector
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [75]:
examples = [
    {'input': "When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention.",
     'output': 'When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.'},

    {'input': "Saying 'He's flexing those dollar bills.' means he's showing off his wealth.",
     'output': 'To utter, "He doth flaunt his gold coins, verily." is to say he parades his riches.'},

    {'input': "The phrase 'That's some tea.' refers to juicy gossip or interesting news.",
     'output': "The saying, 'That's a tale most intriguing.' speaks of a story that piques interest."},

    {'input': "Exclaiming 'It's lit!' means the situation is exciting or fun.",
     'output': "Declaring 'It's a merry revel!' signifies a joyous occasion."},
    {'input': "When someone uses 'Throwing shade', they're subtly expressing disapproval or contempt.",
     'output': 'When one says "Casting aspersions", they art discreetly showing disdain or scorn.'},

    {'input': "The term 'No cap' is used to emphasize that someone is not lying.",
     'output': 'The phrase "In sooth, no falsehood" is uttered to stress that one speaks the truth.'},

    {'input': "To 'Slide into the DMs' means to send someone a direct message, usually with romantic intent.",
     'output': 'To "Venture into private missives" is to send a personal letter, perchance with courtly designs.'},

    {'input': "If someone has 'Ghosted', they've suddenly cut off all communication without explanation.",
     'output': "If one hath 'Vanish'd like a spectre', they've abruptly ceased all discourse without reason."},

    {'input': "The phrase 'On fleek' means that something is perfect or flawless.",
     'output': "The saying 'In finest fettle' signifies that something is in impeccable condition."}
]


In [76]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

In [78]:
example_prompt

PromptTemplate(input_variables=['input', 'output'], template='Input: {input}\nOutput: {output}')

In [77]:
example_selector = NGramOverlapExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    threshold=-1.0
)

dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Translate yon modern utterance into the tongue of the Bard",
    suffix="Modern: {sentence}\nShakespearean:",
    input_variables=["sentence"]
)

In [79]:
dynamic_prompt

FewShotPromptTemplate(input_variables=['sentence'], example_selector=NGramOverlapExampleSelector(examples=[{'input': "When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention.", 'output': 'When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.'}, {'input': "Saying 'He's flexing those dollar bills.' means he's showing off his wealth.", 'output': 'To utter, "He doth flaunt his gold coins, verily." is to say he parades his riches.'}, {'input': "The phrase 'That's some tea.' refers to juicy gossip or interesting news.", 'output': "The saying, 'That's a tale most intriguing.' speaks of a story that piques interest."}, {'input': "Exclaiming 'It's lit!' means the situation is exciting or fun.", 'output': "Declaring 'It's a merry revel!' signifies a joyous occasion."}, {'input': "When someone uses 'Throwing shade', they're subtly expressing disapproval or contempt.", 'output': 'When one says "Casting aspersions", they art

In [80]:
print(dynamic_prompt.format(sentence="Her comment on social media was clearly throwing shade."))

Translate yon modern utterance into the tongue of the Bard

Input: When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention.
Output: When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.

Input: Saying 'He's flexing those dollar bills.' means he's showing off his wealth.
Output: To utter, "He doth flaunt his gold coins, verily." is to say he parades his riches.

Input: The phrase 'That's some tea.' refers to juicy gossip or interesting news.
Output: The saying, 'That's a tale most intriguing.' speaks of a story that piques interest.

Input: Exclaiming 'It's lit!' means the situation is exciting or fun.
Output: Declaring 'It's a merry revel!' signifies a joyous occasion.

Input: When someone uses 'Throwing shade', they're subtly expressing disapproval or contempt.
Output: When one says "Casting aspersions", they art discreetly showing disdain or scorn.

Input: The term 'No cap' is used to emphasize that someone is

In [81]:
print(dynamic_prompt.format(sentence="Everyone noticed her craving for attention at the party."))

Translate yon modern utterance into the tongue of the Bard

Input: Exclaiming 'It's lit!' means the situation is exciting or fun.
Output: Declaring 'It's a merry revel!' signifies a joyous occasion.

Input: To 'Slide into the DMs' means to send someone a direct message, usually with romantic intent.
Output: To "Venture into private missives" is to send a personal letter, perchance with courtly designs.

Input: When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention.
Output: When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.

Input: Saying 'He's flexing those dollar bills.' means he's showing off his wealth.
Output: To utter, "He doth flaunt his gold coins, verily." is to say he parades his riches.

Input: The phrase 'That's some tea.' refers to juicy gossip or interesting news.
Output: The saying, 'That's a tale most intriguing.' speaks of a story that piques interest.

Input: When someone uses 'Throwing shad

In [84]:
example_selector.threshold = -0.25
print(dynamic_prompt.format(sentence="Her comment on social media was clearly throwing shade."))

Translate yon modern utterance into the tongue of the Bard

Input: When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention.
Output: When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.

Input: Saying 'He's flexing those dollar bills.' means he's showing off his wealth.
Output: To utter, "He doth flaunt his gold coins, verily." is to say he parades his riches.

Input: The phrase 'That's some tea.' refers to juicy gossip or interesting news.
Output: The saying, 'That's a tale most intriguing.' speaks of a story that piques interest.

Input: Exclaiming 'It's lit!' means the situation is exciting or fun.
Output: Declaring 'It's a merry revel!' signifies a joyous occasion.

Input: When someone uses 'Throwing shade', they're subtly expressing disapproval or contempt.
Output: When one says "Casting aspersions", they art discreetly showing disdain or scorn.

Input: The term 'No cap' is used to emphasize that someone is

In [85]:
example_selector.threshold = -1
print(dynamic_prompt.format(sentence="The maiden ghosted me post our rendezvous."))

Translate yon modern utterance into the tongue of the Bard

Input: The phrase 'On fleek' means that something is perfect or flawless.
Output: The saying 'In finest fettle' signifies that something is in impeccable condition.

Input: The phrase 'That's some tea.' refers to juicy gossip or interesting news.
Output: The saying, 'That's a tale most intriguing.' speaks of a story that piques interest.

Input: The term 'No cap' is used to emphasize that someone is not lying.
Output: The phrase "In sooth, no falsehood" is uttered to stress that one speaks the truth.

Input: When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention.
Output: When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.

Input: Saying 'He's flexing those dollar bills.' means he's showing off his wealth.
Output: To utter, "He doth flaunt his gold coins, verily." is to say he parades his riches.

Input: Exclaiming 'It's lit!' means the situation is e

## `SemanticSimilarityExampleSelector` Overview

- Specializes in finding examples that match input semantics.

- Utilizes embeddings & vector store for semantic matching.

🔑 **Key Attributes:**

1. **vectorstore**: Stores embeddings of examples.
2. **k**: Number of examples to select (default is 4).

3. **example_keys**: Filters for specific example elements.

4. **input_keys**: Filters for specific input variables.

🔧 **Main Methods:**

1. **`add_example(example)`**:
   - Inserts a new example into `vectorstore`.

   - Constructs a query string based on `input_keys`.

2. **`select_examples(input_variables)`**:

   - Finds examples matching input semantics.

   - Constructs query from input, searches in `vectorstore`.

   - Filters results with `example_keys` if given.

3. **`from_examples(...)`**:

   - Class method to initialize an instance.

   - Sets up `vectorstore` with examples and embeddings.

💡 **How It Works**:

- Employs embeddings for semantic understanding.

- Queries `vectorstore` for semantically close examples.

- Can tailor results based on `example_keys`.

# Initializing the Semantic Similarity Example Selector

To select examples based on their semantic similarity, we use the `SemanticSimilarityExampleSelector`. Here's how it's set up:

- **Examples**: A list of available examples for selection.

- **Embedding Class**: Utilizes `OpenAIEmbeddings()` to generate embeddings, which help measure the semantic similarity between examples.

- **Vector Store**: Employs the `FAISS` class to store these embeddings and perform similarity searches.

- **Number of Examples**: Determines how many examples to produce, set here as `k=4`.






In [86]:
examples

[{'input': "When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention.",
  'output': 'When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.'},
 {'input': "Saying 'He's flexing those dollar bills.' means he's showing off his wealth.",
  'output': 'To utter, "He doth flaunt his gold coins, verily." is to say he parades his riches.'},
 {'input': "The phrase 'That's some tea.' refers to juicy gossip or interesting news.",
  'output': "The saying, 'That's a tale most intriguing.' speaks of a story that piques interest."},
 {'input': "Exclaiming 'It's lit!' means the situation is exciting or fun.",
  'output': "Declaring 'It's a merry revel!' signifies a joyous occasion."},
 {'input': "When someone uses 'Throwing shade', they're subtly expressing disapproval or contempt.",
  'output': 'When one says "Casting aspersions", they art discreetly showing disdain or scorn.'},
 {'input': "The term 'No cap' is used to emphasize 

In [87]:
example_prompt

PromptTemplate(input_variables=['input', 'output'], template='Input: {input}\nOutput: {output}')

In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=4,
)

similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Translate yon modern utterance into the tongue of the Bard",
    suffix="Modern: {sentence}\nShakespearean:",
    input_variables=["sentence"]
)

In [93]:
print(similar_prompt.format(sentence="What does it mean when someone says they're 'Throwing shade' at another?"))

Translate yon modern utterance into the tongue of the Bard

Input: When someone uses 'Throwing shade', they're subtly expressing disapproval or contempt.
Output: When one says "Casting aspersions", they art discreetly showing disdain or scorn.

Input: When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention.
Output: When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.

Input: If someone has 'Ghosted', they've suddenly cut off all communication without explanation.
Output: If one hath 'Vanish'd like a spectre', they've abruptly ceased all discourse without reason.

Input: The phrase 'That's some tea.' refers to juicy gossip or interesting news.
Output: The saying, 'That's a tale most intriguing.' speaks of a story that piques interest.

Modern: What does it mean when someone says they're 'Throwing shade' at another?
Shakespearean:


In [94]:
print(similar_prompt.format(sentence="Someone told me my style was reminiscent of a 'snack'. What could they be hinting at?"))


Translate yon modern utterance into the tongue of the Bard

Input: When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention.
Output: When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.

Input: The phrase 'That's some tea.' refers to juicy gossip or interesting news.
Output: The saying, 'That's a tale most intriguing.' speaks of a story that piques interest.

Input: When someone uses 'Throwing shade', they're subtly expressing disapproval or contempt.
Output: When one says "Casting aspersions", they art discreetly showing disdain or scorn.

Input: If someone has 'Ghosted', they've suddenly cut off all communication without explanation.
Output: If one hath 'Vanish'd like a spectre', they've abruptly ceased all discourse without reason.

Modern: Someone told me my style was reminiscent of a 'snack'. What could they be hinting at?
Shakespearean:


In [95]:
print(similar_prompt.format(sentence="I overheard someone mention they felt 'shook' after watching a movie. What emotion were they expressing?"))


Translate yon modern utterance into the tongue of the Bard

Input: If someone has 'Ghosted', they've suddenly cut off all communication without explanation.
Output: If one hath 'Vanish'd like a spectre', they've abruptly ceased all discourse without reason.

Input: Exclaiming 'It's lit!' means the situation is exciting or fun.
Output: Declaring 'It's a merry revel!' signifies a joyous occasion.

Input: When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention.
Output: When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.

Input: When someone uses 'Throwing shade', they're subtly expressing disapproval or contempt.
Output: When one says "Casting aspersions", they art discreetly showing disdain or scorn.

Modern: I overheard someone mention they felt 'shook' after watching a movie. What emotion were they expressing?
Shakespearean:
